# Linear Bayesian Modelling with Pyro

Welcome to our tutorial notebook on building a simple linear Bayesian model with Pyro! In this tutorial, you will learn the fundamentals of Bayesian modeling and how to implement it using Pyro, a powerful probabilistic programming framework. By the end of this tutorial, you will have a solid understanding of how to define and train a Bayesian model, and how to make probabilistic predictions using posterior samples. Get ready to dive into the world of Bayesian inference and unlock the full potential of your data with Pyro!

## Sections 
0. Setup
1. Defining a Model
2. Defining a Guide
3. Defining a Trainingloop
4. Inference
5. Bonus: SVI vs. MCMC/NUTS
6. Bonus: Comparison with RidgeRegression

## 0. Setup
In this section, we will set up the environment and install the necessary dependencies to run our Bayesian modeling tutorial. We will guide you through the process of installing Pyro and other required libraries, as well as importing the necessary modules and preparing the dataset. By the end of this section, you will have a fully configured environment ready for Bayesian modeling.

In [ ]:
# Installing requirements from the requirements.txt file; the file was created from the poetry.lock with
# poetry export -f requirements.txt --output requirements.txt --without-hashes
!pip3 install -r requirements.txt

In [ ]:
# Import pandas and numpy for data handling
import pandas as pd
import numpy as np

# Import torch and pyro for modelling and inference
import torch
from torch import nn
import pyro
from pyro.nn import PyroSample, PyroParam, PyroModule
import pyro.distributions as dist
from pyro import poutine

# Import plotly to visualize the inference
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Utility
from typing import List
from tqdm import tqdm

In [ ]:
# Loading the dependent and independent features
media = pd.read_parquet("./assets/media_prepped.parquet")
sales = pd.read_parquet("./assets/sales_prepped.parquet")

# Convert the input data to tensors
X = None
y = None

Note: Maybe add some EDA here.

## 1. Defining a Model
In this section, we will delve into the heart of Bayesian modeling by defining our probabilistic model. You will learn how to express your prior beliefs about the parameters of the model using probability distributions. We will walk you through the process of defining the likelihood function, specifying the dependencies between variables, and constructing the model using Pyro's intuitive syntax. By the end of this section, you will have a clear understanding of how to build a Bayesian model in Pyro.

In [ ]:
# Define a PyroModel for predicting the revenue
# per product from the media spend
class RevenueModel(PyroModule):
    def __init__(
        self,
        media_channels: int = 7,
        sales_products: int = 2,
        bias_lower_bound: List[float] = [0.] * 2,
        bias_upper_bound: List[float] = [10_000.] * 2
    ):
        super().__init__()

        # Initialize a linear torch layer as a pyro module.
        # Keep in mind, that the linear layer has in_features, which
        # are equal to the number of sales channels and out_features
        # equal to the number of products
        self.linear_layer = None

        # Set the weight of the linear layer to be sampled from a PyroSamle distributions
        # Potential prior distributions include a folded normal distribution with prior
        # mu one and sigma 0.5 (feel free to experiment though!)
        self.linear_layer.weight = None
        
        # Set the bias of the linear layer to be sampled from a PyroSample
        # Distribution. Potential prior distributions include a uniform distribution
        # between the minimum and the maximum of the two target variables.
        self.linear_layer.bias = None

        # Assign input shapes to instance attributes for later usage
        self.sales_products = sales_products
        self.media_channels = media_channels
        
    def forward(self, X, y=None):
        # Get the expected value of y, from the
        # current state of the linear layer
        y_hat = None # TODO

        noise = None # TODO

        # Open a pyro plate for the 2 samples per row in y
        # You can use any name, but make sure, that it is unique
        # (This is only required since we have multiple dependent
        # variables (i.e. 2 products)). If you only have a single
        # dependent variable, you don't need this first plate.
        with None: # TODO

            # Open a nested plate for the number of rows in your
            # input. The idea here is, that the true y is sampled len(X)
            # times, with expected value y_hat and a sigma, that is sampled
            # per observation.
            # Again, feel free to use any name you want, just make sure, that
            # it is unique.
            with None: # TODO
                # Sample sigma from a pyro.sample.
                # Keep in mind, that the each observation
                # has an independent noise component to it.
                # So the distribution should be '.expand'ed to the
                # shape of y_hat.

                # Sample observations from a suitable distribution.
                # Potential distributions include a normal distribution of
                # mu y_hat and sigma.
                pyro.sample(None) # TODO

        return y_hat

In [ ]:
# Visualize the model
pyro.render_model(RevenueModel(), model_args=(X, y), render_params=True, render_distributions=True)

In [ ]:
# The below trace enables you to see the tensor shapes in terms of batch dimension and
# and event_dimension. This is not critical to do, but can help with debugging.
# Note, how the number of dimensions on the right hand site of the pipe (|) is equal 
# to the number of dimensions we expclicitly reserved for event dimensions with
# .to_event(n_event_dims).
trace = poutine.trace(RevenueModel()).get_trace(X, y)
trace.compute_log_prob()
print(trace.format_shapes())

## 2. Defining a Guide
In Bayesian modeling, a guide is a probability distribution that approximates the posterior distribution of the model's parameters given the observed data. For the sake of brevity during the workshop, we will use an autoguide to train this model.

We are going to define our own guide in the next part of this workshop, but if you are interested, and would like to explore more about guides in Pyro, you are of course very welcome to expand this section.

In [ ]:
class RevenueGuide(pyro.infer.autoguide.AutoDiagonalNormal):
    pass

## 3. Defining a Trainingloop
Training a Bayesian model involves updating the guide's parameters to fit the posterior distribution to the observed data. In this section, we will show you how to define a training loop that iterates over the dataset, performs parameter updates, and tracks the model's performance. You will learn about gradient descent optimization, the use of Pyro's stochastic variational inference (SVI) module, and how to monitor the convergence of your model. By the end of this section, you will be ready to train your Bayesian model effectively.

In [ ]:
# Clear the param store before a new training run
# (good practice)
pyro.clear_param_store()

# Initialize your guide and model
model = RevenueModel()
guide = None # TODO

# Initialize your optimizer and loss
optimizer = None # TODO
loss = None # TODO

# Initialize the SVI
svi = pyro.infer.SVI(
    model=None, # TODO
    guide=None, # TODO
    optim=None, # TODO
    loss=None, # TODO
)

# Define the training loop
EPOCHS = None # TODO

# Define the loop as taking a 'step' in each Epoch
# Feel free to print and/or store the loss as you'd
# like.
losses = []
for epoch in tqdm(range(EPOCHS), total=EPOCHS):
    pass # TODO

# Use plotly, or any other visualization library to show the decrease of the loss
px.line(losses)

## 4. Inference
Once the model is trained, we can use it to make predictions. In this section, we will guide you through the process of using the trained model and guide to perform inference on new data points. You will learn how to draw posterior samples, compute predictive distributions, and visualize the results. By the end of this section, you will have the skills to leverage the power of Bayesian inference in your own projects.

In [ ]:
# Initialize the predicitive class to sample from the fitted model.
# Initialize it with the model, the guide and the number of samples that you want to draw for each variable
predictive = pyro.infer.Predictive(
    model=None, # TODO
    guide=None, # TODO
    num_samples=None # TODO
)

# To get the samples per parameter (including the predicted observations),
# pass the input tensor you want to sample for to the sampler.
# The output will be a dictionary with the parameter names as keys and the
# parameter's samples as values.
samples = None # TODO

# Fetch the estimated coefficients for visualization.
# Pyro may add arbitrary dimensions during sampling, so make
# sure to reshape the sampled tensor intor a suitable shape
beta_hat = None # TODO
beta_hat_prime = None # TODO

# Visualize the distribution of samples with plotly or any other library you like
fig = make_subplots(rows=2, cols=7)

for row, product in enumerate(beta_hat_prime, start=1):
    for column, coefficient_samples in enumerate(product, start=1):
        fig.add_trace(
            go.Histogram(x=coefficient_samples, name=f"{sales.columns[row - 1]} - {media.columns[column - 1]}"), row=row, col=column,
        )
        
fig.show()